Goal of analysis script is to pull in txt files of annotated USVs and generate summary statistics

Import modules

In [13]:
import pandas as pd
import numpy as np
from tkinter import filedialog
from tkinter import *
import re
import os
from scipy import stats

# visualization
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Open dialog box and prompt user to select directory containing the data files 

In [15]:
root = Tk()
root.directory = filedialog.askdirectory()
print (root.directory)

C:/Users/Schindler/Documents/Schindler_Lab/Data/USVs/CPA_pair_exp/3x_pair


Create a path name for each data file

In [16]:
path_names = []
files = os.listdir(root.directory)
for file in files: 
        path_names.append(root.directory + "/" + file)

path_names[0]

'C:/Users/Schindler/Documents/Schindler_Lab/Data/USVs/CPA_pair_exp/3x_pair/527.Table.1.selections.cagepair.txt'

Loop through path names to create pandas data table from excel file. Get animal number and session type from path name (animal number is part of file name)

In [17]:
data_table = pd.DataFrame()

i = 0

for path in path_names:
    data = pd.read_table(path)
    data_table_int = pd.DataFrame(data = data)
    
    name = re.search("\d\d\d", path).group(0)
    name_mult = [name] * len(data_table_int)
    session = re.search("base|cagepair|CPApair", path).group(0)
    session = [session] * len(data_table_int)
    i = i + 1
    
    data_table_int = data_table_int.assign(Animal = name_mult)
    data_table_int = data_table_int.assign(Session = session)
    
    data_table = data_table.append(data_table_int, ignore_index=True)

data_table.head()

,Selection,View,Channel,Begin Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Delta Time (s),Delta Freq (Hz),Avg Power Density (dB FS),Annotation,Animal,Session
0,1,Spectrogram 1,1,18.929297,18.934698,74332.5,78276.7,0.005,3944.2,-44.4,high slug,527,cagepair
1,2,Spectrogram 1,1,24.758961,24.769763,10012.1,13046.1,0.011,3034.0,-41.6,low slug,527,cagepair
2,8,Spectrogram 1,1,174.353273,174.357636,34706.7,41988.2,0.004,7281.5,-41.5,bbc high,527,cagepair
3,12,Spectrogram 1,1,342.886783,342.892601,16874.1,53888.7,0.006,37014.6,-39.1,bbc long,527,cagepair
4,14,Spectrogram 1,1,393.056112,393.061930,35194.2,50970.9,0.006,15776.7,-39.1,bbc high,527,cagepair


In [18]:
data_table.drop(['Selection', 'View', 'Channel'], axis=1, inplace = True)

In [19]:
data_table.head()

,Begin Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Delta Time (s),Delta Freq (Hz),Avg Power Density (dB FS),Annotation,Animal,Session
0,18.929297,18.934698,74332.5,78276.7,0.005,3944.2,-44.4,high slug,527,cagepair
1,24.758961,24.769763,10012.1,13046.1,0.011,3034.0,-41.6,low slug,527,cagepair
2,174.353273,174.357636,34706.7,41988.2,0.004,7281.5,-41.5,bbc high,527,cagepair
3,342.886783,342.892601,16874.1,53888.7,0.006,37014.6,-39.1,bbc long,527,cagepair
4,393.056112,393.061930,35194.2,50970.9,0.006,15776.7,-39.1,bbc high,527,cagepair


Organize call types into 

In [20]:
USV_groups = data_table.replace('mid slug', 'low slug', inplace = True)
USV_groups = data_table.replace(['bbc wide', 'bbc long', 'bbc high', 'bbc low'], 'bbc')
USV_groups['Annotation'].unique()

array(['high slug', 'low slug', 'bbc', 'high multi', 'low multi', 'radar'],
      dtype=object)

In [21]:
USV_groups.head()

,Begin Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Delta Time (s),Delta Freq (Hz),Avg Power Density (dB FS),Annotation,Animal,Session
0,18.929297,18.934698,74332.5,78276.7,0.005,3944.2,-44.4,high slug,527,cagepair
1,24.758961,24.769763,10012.1,13046.1,0.011,3034.0,-41.6,low slug,527,cagepair
2,174.353273,174.357636,34706.7,41988.2,0.004,7281.5,-41.5,bbc,527,cagepair
3,342.886783,342.892601,16874.1,53888.7,0.006,37014.6,-39.1,bbc,527,cagepair
4,393.056112,393.061930,35194.2,50970.9,0.006,15776.7,-39.1,bbc,527,cagepair


In [22]:
counts = USV_groups.groupby(['Session'])['Annotation'].count()
counts

Session
CPApair     478
cagepair     74
Name: Annotation, dtype: int64

In [23]:
writer = pd.ExcelWriter('USV_counts.xlsx', engine='xlsxwriter')
counts.to_excel(writer, sheet_name='Sheet1')
writer.save()

In [24]:
writer = pd.ExcelWriter('USV_annot.xlsx', engine='xlsxwriter')
USV_groups.to_excel(writer, sheet_name='Sheet1')
writer.save()